In [ ]:
import pandas as pd
import requests

In [12]:
urlheader = {
      "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36",
      "authority": "www.nseindia.com",
      "scheme":"https"
    }
proxies = {
  "http": None,
  "https": None,
}   
nse_api = "https://www.nseindia.com/api/option-chain-indices?"
params="symbol=BANKNIFTY"

url_encoded=nse_api + params

req = requests.get(url_encoded, headers=urlheader,verify = False, proxies=proxies)

C:\Users\gileadb\Anaconda\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [16]:
req.json()['records']['data']

[{'strikePrice': 13900,
  'expiryDate': '09-Apr-2020',
  'CE': {'strikePrice': 13900,
   'expiryDate': '09-Apr-2020',
   'underlying': 'BANKNIFTY',
   'identifier': 'OPTIDXBANKNIFTY09-04-2020CE13900.00',
   'openInterest': 2,
   'changeinOpenInterest': 1,
   'pchangeinOpenInterest': 100,
   'totalTradedVolume': 12,
   'impliedVolatility': 0,
   'lastPrice': 6074.15,
   'change': 1219.3499999999995,
   'pChange': 25.116379665485695,
   'totalBuyQuantity': 2440,
   'totalSellQuantity': 2440,
   'bidQty': 20,
   'bidprice': 5239.2,
   'askQty': 20,
   'askPrice': 6083.3,
   'underlyingValue': 19913.6},
  'PE': {'strikePrice': 13900,
   'expiryDate': '09-Apr-2020',
   'underlying': 'BANKNIFTY',
   'identifier': 'OPTIDXBANKNIFTY09-04-2020PE13900.00',
   'openInterest': 1992,
   'changeinOpenInterest': -1127,
   'pchangeinOpenInterest': -36.13337608207759,
   'totalTradedVolume': 4409,
   'impliedVolatility': 0,
   'lastPrice': 0.05,
   'change': -2.1500000000000004,
   'pChange': -97.727272

In [18]:
ce_list = []
pe_list = []
data = req.json()['records']['data']
for exp_week in data:
    if 'CE' in exp_week:
        ce_list.append(exp_week['CE'])
    else:
        pe_list.append(exp_week['PE'])

In [19]:
ce_df = pd.DataFrame(ce_list)
pe_df = pd.DataFrame(pe_list)

In [27]:
len(ce_df), len(pe_df)

(756, 138)

In [22]:
# ce_df.strikePrice.unique(),pe_df.strikePrice.unique()

In [28]:
ce_df['expiryDate'] = pd.to_datetime(ce_df['expiryDate'])
pe_df['expiryDate'] = pd.to_datetime(pe_df['expiryDate'])

In [31]:
filtered_ce_data = ce_df[['identifier','strikePrice','expiryDate','lastPrice','openInterest']].sort_values(['expiryDate','strikePrice'])
filtered_pe_data = pe_df[['identifier','strikePrice','expiryDate','lastPrice','openInterest']].sort_values(['expiryDate','strikePrice'])
print('-----------------------Last 5 CE records---------------------')
print(filtered_ce_data.head())
print('-----------------------Last 5 PE records---------------------')
print(filtered_pe_data.tail())

-----------------------Last 5 CE records---------------------
                             identifier  strikePrice expiryDate  lastPrice  \
0   OPTIDXBANKNIFTY09-04-2020CE13900.00        13900 2020-04-09    6074.15   
6   OPTIDXBANKNIFTY09-04-2020CE14000.00        14000 2020-04-09    6457.80   
9   OPTIDXBANKNIFTY09-04-2020CE14100.00        14100 2020-04-09       0.00   
15  OPTIDXBANKNIFTY09-04-2020CE14200.00        14200 2020-04-09       0.00   
17  OPTIDXBANKNIFTY09-04-2020CE14300.00        14300 2020-04-09       0.00   

    openInterest  
0              2  
6              1  
9              0  
15             0  
17             0  
-----------------------Last 5 PE records---------------------
                             identifier  strikePrice expiryDate  lastPrice  \
9   OPTIDXBANKNIFTY28-05-2020PE16200.00        16200 2020-05-28          0   
10  OPTIDXBANKNIFTY28-05-2020PE16800.00        16800 2020-05-28          0   
12  OPTIDXBANKNIFTY28-05-2020PE17000.00        17000 2020-0

In [33]:
# writing master Df to csv
ce_df.to_csv('ce_data.csv',encoding='utf-8',index=False)
pe_df.to_csv('pe_data.csv',encoding='utf-8',index=False)